In [1]:
from transformers import AutoImageProcessor, ResNetModel
import torch
from datasets import load_dataset

In [2]:
dataset = load_dataset("imagenet-1k")
image = dataset["test"]["image"][0]

from transformers import AutoImageProcessor
image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
inputs = image_processor(image, return_tensors="pt")

model = ResNetModel.from_pretrained("microsoft/resnet-50")



with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

/home/mheuillet/Desktop/robust_training/.venv/lib/python3.8/site-packages/datasets/load.py:1486: FutureWarning: The repository for imagenet-1k contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/imagenet-1k
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [14]:
inputs['pixel_values'].shape

torch.Size([1, 3, 224, 224])